## Pre Proccessing


In [ ]:
import tensorflow as tf
import numpy as np
from sklearn.model_selection import train_test_split

# Load MNIST dataset
(X_train, y_train), (X_test, y_test) = tf.keras.datasets.mnist.load_data()

train_filter = (y_train == 0) | (y_train == 1)
test_filter = (y_test == 0) | (y_test == 1)

X_train_binary = X_train[train_filter]
y_train_binary = y_train[train_filter]
X_test_binary = X_test[test_filter]
y_test_binary = y_test[test_filter]

X_train_binary = X_train_binary / 255.0
X_test_binary = X_test_binary / 255.0

X_train_binary = X_train_binary.reshape(-1, 28*28)
X_test_binary = X_test_binary.reshape(-1, 28*28)

y_train_binary = np.where(y_train_binary == 0, 0, 1)
y_test_binary = np.where(y_test_binary == 0, 0, 1)

X_val_binary, X_test_binary, y_val_binary, y_test_binary = train_test_split(X_test_binary, y_test_binary, test_size=0.6, random_state=42)
X_train_binary.shape, y_train_binary.shape, X_val_binary.shape, y_val_binary.shape, X_test_binary.shape, y_test_binary.shape

: 

In [ ]:
hidden_layers = int(input("Enter the number of hidden layers: "))
neurons_per_layer = []
for i in range(hidden_layers):
    neurons_per_layer.append(int(input(f"Enter the number of neurons for layer {i+1}: ")))

activation_function = input("Enter the activation function for hidden layers (sigmoid/relu): ").strip().lower()

In [ ]:
def derivative_activation_function(z, func):
    if func == 'sigmoid':
        sig = 1 / (1 + np.exp(-z))
        return sig * (1 - sig)
    elif func == 'relu':
        return np.where(z > 0, 1, 0)
    elif func == 'tanh':
        return 1 - np.tanh(z)**2
    else:
        raise ValueError("Unsupported activation function")

In [ ]:
def forward_propagation(X, weights, biases, activation_function):
    a = X
    zs = []
    activations = [X]
    for l in range(len(weights)):
        z = weights[l] @ a + biases[l]
        zs.append(z)
        if (l == len(weights) - 1):
            a = 1 / (1 + np.exp(-z))
        else:
            if activation_function == 'sigmoid':
                a = 1 / (1 + np.exp(-z))
            elif activation_function == 'relu':
                a = np.maximum(0, z)
            elif activation_function == 'tanh':
                a = np.tanh(z)
            else:
                raise ValueError("Unsupported activation function")
        activations.append(a)
    return zs, activations

In [ ]:
def back_propagation(zs, activations, weights, y, activation_function):
    deltas = [None] * len(weights)
    deltas[-1] = activations[-1] - y
    for l in range(len(deltas) - 2, -1, -1):
        deltas[l] = (weights[l + 1].T @ deltas[l + 1]) * derivative_activation_function(zs[l], activation_function)
    weights_gradients = []
    biases_gradients = []
    for l in range(len(deltas)):
        weights_gradients.append(deltas[l] @ activations[l].T)
        biases_gradients.append(np.sum(deltas[l], axis=1, keepdims=True))
    return weights_gradients, biases_gradients

In [ ]:
def update_parameters(weights, biases, weights_gradients, biases_gradients, learning_rate):
    for l in range(len(weights)):
        weights[l] -= learning_rate * weights_gradients[l]
        biases[l] -= learning_rate * biases_gradients[l]
    return weights, biases

In [ ]:
def train_neural_network(X_train, y_train, X_val, y_val, hidden_layers, neurons_per_layer, activation_function, epochs=1000, learning_rate=0.01, batch_size=64):
    input_size = X_train.shape[1]
    output_classes = 1
    weights = []
    biases = []
    layers = [input_size] + neurons_per_layer + [output_classes]
    train_losses = []
    train_accuracys = []
    validation_losses = []
    validation_accuracys = []
    for i in range(1, hidden_layers + 2):
        w = np.random.randn(layers[i], layers[i - 1]) * 0.01
        b = np.zeros((layers[i], 1))
        weights.append(w)
        biases.append(b)
    # show initial accuracy
    _, train_activations = forward_propagation(X_train.T, weights, biases, activation_function)
    train_predictions = (train_activations[-1] > 0.5).astype(int)
    train_accuracy = np.mean(train_predictions.flatten() == y_train_binary) * 100
    print(f"Initial Accuracy: {train_accuracy:.2f}%")
    for epoch in range(epochs):
        permutation = np.random.permutation(X_train.shape[0])
        X_shuffled = X_train[permutation]
        y_shuffled = y_train[permutation]
        for i in range(0, X_train.shape[0], batch_size):
            X_batch = X_shuffled[i:i + batch_size].T
            y_batch = y_shuffled[i:i + batch_size].reshape(1, -1)
            zs, activations = forward_propagation(X_batch, weights, biases, activation_function)
            weights_gradients, biases_gradients = back_propagation(zs, activations, weights, y_batch, activation_function)
            weights, biases = update_parameters(weights, biases, weights_gradients, biases_gradients, learning_rate)
        # show accuracy per epoch
        _, train_activations = forward_propagation(X_train.T, weights, biases, activation_function)
        train_predictions = (train_activations[-1] > 0.5).astype(int)
        train_accuracy = np.mean(train_predictions.flatten() == y_train) * 100
        train_accuracys.append(train_accuracy)
        _, validation_activations = forward_propagation(X_val.T, weights, biases, activation_function)
        validation_predictions = (validation_activations[-1] > 0.5).astype(int)
        validation_accuracy = np.mean(validation_predictions.flatten() == y_val) * 100
        validation_accuracys.append(validation_accuracy)
        print(f"Epoch {epoch + 1}, Accuracy: {train_accuracy:.6f}%, Validation Accuracy: {validation_accuracy:.6f}%")
        # show loss per epoch
        train_loss = -np.mean(y_train * np.log(train_activations[-1] + 1e-8) + (1 - y_train) * np.log(1 - train_activations[-1] + 1e-8))
        train_losses.append(train_loss)
        validation_loss = -np.mean(y_val * np.log(validation_activations[-1] + 1e-8) + (1 - y_val) * np.log(1 - validation_activations[-1] + 1e-8))
        validation_losses.append(validation_loss)
        print(f"Epoch {epoch + 1}, Loss: {train_loss:.6f}, Validation Loss: {validation_loss:.6f}")
    return weights, biases, train_losses, train_accuracys, validation_losses, validation_accuracys

In [ ]:
def train_and_evaluate(params, X_train, y_train, X_val, y_val):
    hidden_layers = params["hidden_layers"]
    neurons = params["neurons"]
    activation = params["activation"]
    learning_rate = params["learning_rate"]
    batch_size = params["batch_size"]
    epochs = params["epochs"]

    weights, biases, _, train_accs, _, val_accs = train_neural_network(
        X_train, y_train,
        X_val, y_val,
        hidden_layers,
        neurons,
        activation_function=activation,
        epochs=epochs,
        learning_rate=learning_rate,
        batch_size=batch_size
    )

    
    best_val_acc = max(val_accs)

    return best_val_acc


In [ ]:
import random

def random_search(num_trials, X_train, y_train, X_val, y_val):
    best_acc = 0
    best_params = None
    history = []

    for trial in range(num_trials):
        hidden_layers = random.randint(1, 5)
        neurons = [random.choice([32, 64, 128, 256, 512]) for _ in range(hidden_layers)]

        params = {
            "hidden_layers": hidden_layers,
            "neurons": neurons,
            "activation": random.choice(["relu", "tanh", "sigmoid"]),
            "learning_rate": 10 ** random.uniform(-5, -1),
            "batch_size": random.choice([16, 32, 64, 128]),
            "epochs": random.randint(3, 20)
        }

        print(f"\n[Random Search] Trial {trial+1}")
        print("Params:", params)

        val_acc = train_and_evaluate(params, X_train, y_train, X_val, y_val)
        history.append((params, val_acc))

        print(f"Validation Accuracy: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            best_params = params

    print("\n🏆 Best Random Search Result")
    print("Accuracy:", best_acc)
    print("Params:", best_params)

    return best_params, best_acc, history


In [ ]:
import itertools

def grid_search(X_train, y_train, X_val, y_val):
    hidden_layers_list = [1, 2, 3]
    neurons_list = [[64], [128], [128, 64]]
    activations = ["relu", "tanh"]
    learning_rates = [1e-2, 1e-3]
    batch_sizes = [32, 64]
    epochs_list = [5, 10]

    best_acc = 0
    best_params = None
    history = []

    for hl, neu, act, lr, bs, ep in itertools.product(
        hidden_layers_list,
        neurons_list,
        activations,
        learning_rates,
        batch_sizes,
        epochs_list
    ):
        if len(neu) != hl:
            continue

        params = {
            "hidden_layers": hl,
            "neurons": neu,
            "activation": act,
            "learning_rate": lr,
            "batch_size": bs,
            "epochs": ep
        }

        print("\n[Grid Search] Params:", params)

        val_acc = train_and_evaluate(params, X_train, y_train, X_val, y_val)
        history.append((params, val_acc))

        print(f"Validation Accuracy: {val_acc:.2f}%")

        if val_acc > best_acc:
            best_acc = val_acc
            best_params = params

    print("\n🏆 Best Grid Search Result")
    print("Accuracy:", best_acc)
    print("Params:", best_params)

    return best_params, best_acc, history


In [ ]:
import tensorflow as tf
from tensorflow.keras.applications import mobilenet_v2
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
def extract_features(X_train):
    model = mobilenet_v2.MobileNetV2(weights='imagenet', include_top=False, pooling='avg', input_shape=(32, 32, 3))
    X_train = X_train.reshape(-1, 28, 28)
    X_train = np.expand_dims(X_train, axis=-1)
    resized_images = tf.image.resize(X_train, [32, 32])
    resized_images = tf.repeat(resized_images, 3, axis=-1)
    preprocessed_images = preprocess_input(resized_images)
    features = model.predict(preprocessed_images)
    return features

In [ ]:
extracted_features = extract_features(X_train_binary)
extracted_val_features = extract_features(X_val_binary)
weights, biases, train_losses, train_accuracys, validation_losses, validation_accuracys = train_neural_network(extracted_features, y_train_binary, extracted_val_features, y_val_binary, hidden_layers, neurons_per_layer, activation_function, 100,learning_rate=0.03)

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Input
model = Sequential([
    Input(shape=(extracted_features.shape[1],)),
    *[Dense(neurons, activation=activation_function) for neurons in neurons_per_layer],
    Dense(1, activation='sigmoid')
])
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
model.fit(extracted_features, y_train_binary, epochs=100, batch_size=64)

In [ ]:
def predict(X, weights, biases, activation_function):
    _, activations = forward_propagation(X.T, weights, biases, activation_function)
    predictions = (activations[-1] > 0.5).astype(int)
    return predictions.flatten()

In [ ]:
extracted_test_features = extract_features(X_test_binary)
predictions = predict(extracted_test_features, weights, biases, activation_function)
accuracy = np.mean(predictions == y_test_binary) * 100
accuracy

In [ ]:
def plot_results(train_losses, train_accuracys, validation_losses, validation_accuracys):
    import matplotlib.pyplot as plt
    epochs = range(1, len(train_losses) + 1)
    plt.figure(figsize=(12, 5))
    plt.subplot(1, 2, 1)
    plt.plot(epochs, train_losses, label='Training Loss')
    plt.plot(epochs, validation_losses, label='Validation Loss')
    plt.xlabel('Epochs')
    plt.ylabel('Loss')
    plt.title('Training and Validation Loss')
    plt.legend()
    plt.subplot(1, 2, 2)
    plt.plot(epochs, train_accuracys, label='Training Accuracy')
    plt.plot(epochs, validation_accuracys, label='Validation Accuracy')
    plt.xlabel('Epochs')
    plt.ylabel('Accuracy (%)')
    plt.title('Training and Validation Accuracy')
    plt.legend()
    plt.show()

In [ ]:
plot_results(train_losses, train_accuracys, validation_losses, validation_accuracys)